In [1]:
%matplotlib inline


# Neural Networks

可以使用``torch.nn``构建神经网络，`nn`依赖`autograd`去定义模型与differentiate他们。\
一个``nn.Module``包含了layers，和一个``forward(input)``方法，它返回``output``.

下图是个简单的feed-forward网络
![convnet](./assets/mnist.png)


典型的训练流程如下:
- 定义一个NN，包含了可训练的参数（或者权重）
- 遍历输入数据，并通过NN前向计算。
- 计算loss
- 如BP算法对梯度进行反向传播
- 更新weights，比较典型的方法如：``weight = weight - learning_rate * gradient``


# Define the network

tips:
> ``torch.nn``仅仅支持mini-batches。并不支持单个的样本。
> 例如：``nn.Conv2d``采用一个4D的Tensor ``nSamples x nChannels x Height x Width``。如果只有一个原本，需要使用``input.unsqueeze(0)``去增加一个假的batch维度

其中需要关注的一点：
`autograd.Function`:实现autograd操作的前向和后向定义。每个Tensor操作都至少创建一个Function节点，该节点连接到创建这个Tensor并对其历史进行编码的function。

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只需要继承`nn.Model`，重载`forward`函数，``backward``函数（此函数计算梯度）会使用`autograde`自动定义。

模型的可学习参数由`net.parameters()`返回。


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


测试一个随机数据32x32 input，channels=1。\
ps：如果使用MNIST数据，需要转为32x32格式

In [11]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0406, -0.0002, -0.0547,  0.1304,  0.0782, -0.0203,  0.0090,  0.0217,
          0.0257,  0.0260]], grad_fn=<AddmmBackward>)


清零梯度缓冲区，并随机初始化

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

tips:
> ``torch.nn``仅仅支持mini-batches。并不支持单个的样本。
> 例如：``nn.Conv2d``采用一个4D的Tensor ``nSamples x nChannels x Height x Width``。如果只有一个原本，需要使用``input.unsqueeze(0)``去增加一个假的batch维度


# Loss Function

不同损失函数参考：[loss functions](https://pytorch.org/docs/nn.html#loss-functions)

参考一个简单的：``nn.MSELoss``


In [13]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9550, grad_fn=<MseLossBackward>)


使用`loss`的`.grad_fn`属性去跟踪，可以找到他的计算图：

```

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss
```

如果调用调用``loss.backward()``函数，整个图就是对loss的微分。前提是设置了``requires_grad=True``，将梯度累加到``.grad`` Tensor中。



In [14]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# Backprop（反向传播）

每次的调用 ``loss.backward()``,需要调用`.zero_grad()`将梯度清零，否则梯度会累计。



In [15]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0028, -0.0072, -0.0002,  0.0045,  0.0055, -0.0050])


# 更新weights

The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):
时间中最间的更新规则是SGD：
$$
    weight = weight - learning_rate * gradient
$$

可以简单的用python实现:

```python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
```

实际上，在NN中，会使用不同的算法，如SGD, Nesterov-SGD, Adam, RMSProp等，我们需要一个包：``torch.optim`` ：


In [17]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update